
# concurrent seperation logic
CSL
https://read.seas.harvard.edu/~kohler/class/cs260r-17/brookes16concurrent.pdf

How to mix temporal and separation logic. Statements about heap over time, processes.
Propsotional dynamic logic but with separating conjunction?

owicki and gries 

Higher order separation logic.


# Separation Logic
Pulse
https://www.youtube.com/watch?v=jCqTVSSV_ao  - swami overview of pulse

Wait, DICE? Bootloader?

https://www.youtube.com/watch?v=mBkzziLPOl8  [PLDI'25] PulseCore: An Impredicative Concurrent Separation Logic for Dependently Typed Programs

Skip navigation
Search



Create

1

Avatar image
Pulse: Proof-oriented Programming with Concurrent Separation Logic in F*

separation
linear
dynamic logic
https://www.philipzucker.com/shallow_logic_knuckle/


Maybe instead make it parametric over the heap kind


In [ ]:
import kdrag.theories.set as set_
from kdrag.all import *
class Sep:
    def __init__(self, KeySort : smt.SortRef):
        self.KeySort = KeySort
        key = smt.Const("key", KeySort)
        self.AllocSort = set_.Set(KeySort)
        h,k,l = smt.Consts("h k l", self.AllocSort)
        self.h = h
        self.k = k
        self.l = l
        self.Prop = smt.ArraySort(self.AllocSort, smt.BoolSort())
        self.emp = smt.Lambda([h], h == self.AllocSort.empty)
        p,q,r = smt.Consts("p q r", self.Prop)
        self.sep = kd.define("sep", [p,q], self.Sep(p, q))
        self.alloc = kd.define("alloc", [key], self.Alloc(key))
        self.true = smt.Lambda([h], smt.BoolVal(True))
        self.false = smt.Lambda([h], smt.BoolVal(False))
        #self.wand
    def Lift(self, expr):
        return smt.Lambda([self.h], expr) # Const
    def Alloc(self, key):
        h = self.h
        return smt.Lambda([h], h == smt.Store(self.AllocSort.empty, key, True))
    def Pto(self, key, v):
        h = self.h
        heap = smt.Const("heap", self.KeySort, v.sort())
        return self.And(self.Alloc(key), self.Lift(heap[k] == v))
    def Sep(self, a, b): # multistar *args?
        h = self.h
        k = self.k
        return smt.Lambda([h], kd.QExists([k], k <= h, a[h - k] & b[k]))
    def Wand(self, a, b):
        h = self.h
        k = self.k
        return smt.Lambda([h], kd.QForAll([k], k & h == self.Alloc.empty, a[k], b[k | h]))
    def And(self, *args):
        h = self.h
        return smt.Lambda([h], smt.And([a[h] for a in args]))
    def Or(self, *args):
        h = self.h
        return smt.Lambda([h], smt.Or([a[h] for a in args]))
    def Bool(self, name):
        return smt.Const(name, self.Prop)
    def Bools(self, names):
        return smt.Consts(names, self.Prop)
    def Valid(self, prop):
        h = smt.FreshConst(self.Alloc, prefix="h")
        return smt.ForAll([h], prop[h])


Sep8 = Sep(smt.BitVecSort(8))
p,q = Sep8.Bools("p q")

kd.prove(smt.ForAll([Sep8.h], Sep8.Star(p,q)[Sep8.h] == Sep8.Star(q,p)[Sep8.h]))
kd.prove(smt.ForAll([Sep8.h], Sep8.Star(p,Sep8.emp)[Sep8.h] == p[Sep8.h]))


# sep_assoc

# wand curry

# frame  P -* Q -> P * R -* Q * R

# P /\ P -* Q -> Q

#



|= ForAll(h,
       Lambda(h,
              Exists(k,
                     And(subset(k, h),
                         And(p[setminus(h, k)],
                             Lambda(h,
                                    h == K(BitVec(8), False))[k]))))[h] ==
       p[h])

In [ ]:
import kdrag.theories.logic.sep as sep
from kdrag.all import *
import kdrag.theories.logic.axioms as ax

In [ ]:
x = smt.Int("x")
smt.is_array(smt.Lambda([x], x > 0))

False

In [ ]:
type FuncRef = smt.ArrayRef | smt.QuantifierRef
def domain(f : FuncRef) -> list[smt.SortRef]:
    if isinstance(f, smt.ArrayRef):
        res = []
        i = 0
        try:
            while True:
                res.append(f.domain_n(i))
                i += 1
        except smt.Z3Exception:
            return res
    elif isinstance(f, smt.QuantifierRef) and f.is_lambda():
        return [v.sort() for v in range(f.num_vars())]
    else:
        raise TypeError(f"Expected ArrayRef or Lambda, got {f}")

def range_(f : FuncRef) -> smt.SortRef:
    if isinstance(f, smt.ArrayRef):
        return f.range()
    elif isinstance(f, smt.QuantifierRef) and f.is_lambda():
        return f.body().sort()
    else:
        raise TypeError(f"Expected ArrayRef or Lambda, got {f}")

In [ ]:
import kdrag.theories.logic.sep as sep
from kdrag.all import *
import kdrag.theories.logic.axioms as ax
Sep8 = sep.make_sep(smt.BitVecSort(8))
p,q,r = kd.FreshVars("p q r", Sep8.PropSort)
l = kd.Lemma(Sep8.sep(p,q) == Sep8.sep(q,p))
l.unfold(Sep8.sep)
l.rw(ax.ext(Sep8.AllocSort, smt.BoolSort()))
l.auto()
l.qed().forall([p,q])

|= ForAll([p!42, q!43], sep(p!42, q!43) == sep(q!43, p!42))

In [1]:
import kdrag.theories.logic.sep as sep
from kdrag.all import *
import kdrag.theories.logic.axioms as ax
Sep8 = sep.make_sep(smt.BitVecSort(8))
p,q,r = kd.FreshVars("p q r", Sep8.PropSort)
l = kd.Lemma(Sep8.sep(p,Sep8.emp) == p)
l.unfold(Sep8.sep, Sep8.emp.decl())
l.rw(ax.ext(Sep8.AllocSort, smt.BoolSort()))
l.auto()
l.qed().forall([p])

|= ForAll(p!42, sep(p!42, emp) == p!42)

# separation
https://www.philipzucker.com/executable-seperation/

appell paper https://www.cs.princeton.edu/~appel/papers/bringing-order.pdf/

"And here's the paper with a nice clean separation logic semantics, with explicit connections to Kripke frames: https://www.cs.princeton.edu/~appel/papers/bringing-order.pdf/. The corresponding models of relevance logic are Routley-Myers models. The distinctive feature of the signature is the same in both cases (a three-place relation R), and the semantics for the relevant conditional are exactly Cao-Culler-Appel's weak semantics for the wand. But there are a lot of other constraints that are different (Routley-Meyers assumes a distinguished element 0 s.t. R0ab iff a<b, so they don't have < as an extra part of a model's signature. Cao-Culler-Appel makes < part of the signature. Cao-Culler-Appel assumes that Rabc implies Rbac, and this isn't built in to all relevance logics)."

https://iris-project.org/tutorial-pdfs/iris-lecture-notes.pdf

In [ ]:
from kdrag.all import *
import kdrag.theories.set as set_

Addr = smt.DeclareSort("Addr") # smt.IntSort()
AddrSet = set_.Set(Addr)
addrs = smt.Const("addrs", AddrSet)
Prop = kd.NewType('Prop', smt.ArraySort(AddrSet, smt.BoolSort()))

p,q,r = smt.Consts("p q r", Prop)

Emp = Prop(smt.Lambda([addrs], addrs == ValidAddr.empty))
emp = kd.define("emp", [], Emp)

def heap(S):
    return smt.Const("heap", smt.ArraySort(smt.IntSort(), S))

#def PTo(p, x):
#    return Prop(smt.Lambda([addrs], smt.And(addrs == smt.Store(ValidAddr.empty, p, True) , heap(x.sort())[p] == x)))

n = smt.Int("n")
def Alloc(p):
    return Prop(smt.Lambda([addrs], addrs == smt.Store(ValidAddr.empty, p, True)))
alloc = kd.define("alloc", [n], Alloc(n))

# needs to be polymorhic. On second argument :(
#pto = kd.define("pto", ["p", "x"], PTo(p, x))

def And(*args):
    return Prop(smt.Lambda([addrs], smt.And(*[arg.val(addrs) for arg in args])))

and_ = kd.define("and_", [p,q], And(p, q))

def Or(*args):
    return Prop(smt.Lambda([addrs], smt.Or(*[arg.val(addrs) for arg in args])))

or_ = kd.define("or_", [p,q], Or(p, q))

def Sep(a, b):
    addrs1, addrs2 = smt.Consts("addrs1 addrs2", ValidAddr) # Freshconst?
    return Prop(smt.Lambda([addrs], smt.Exists([addrs1, addrs2], smt.And(addrs1 & addrs2 == ValidAddr.empty, addrs1 | addrs2 == addrs, a.val[addrs1], b.val[addrs2] ))))

sep = kd.define("sep", [p,q], Sep(p, q))

def Wand(p,q):
    


def Valid(p):
    return smt.Exists([addrs], p.val[addrs])

valid = kd.define("valid", [p], Valid(p))

kd.prove(Valid(emp), unfold=1)    #)y=emp.defn)
kd.prove(Valid(Sep(Emp, Emp)))
kd.prove(valid(sep(emp,emp)), unfold=1)

x = smt.BitVec("x", 32)

#smt.simplify(Valid(Sep(PTo(1, x), PTo(2, x))))
kd.prove(valid(sep(alloc(1), alloc(2))), unfold=1)
#kd.prove(valid(sep(alloc(n), alloc(n))), unfold=1)


IndentationError: expected an indented block after function definition on line 44 (2905299801.py, line 47)

https://en.wikipedia.org/wiki/Bunched_logic

https://en.wikipedia.org/wiki/Relevance_logic
https://plato.stanford.edu/entries/logic-relevance/


In [ ]:
from kdrag.all import *
import functools
@functools.cache
def MultiSet(T): # Bag
    S = kd.NewType("Bag", smt.ArraySort(T, smt.IntSort()), pred=lambda x: smt.ForAll([k], k[n] >= 0))
    def Sum(a, b):
        return smt.Map(plus, a.val, b.val)
    #kd.notation.add.define([])
    return S
n,m = Ints("n m")
plus = (n + m).decl()
def Sum(a, b):
    return smt.Map(plus, a.val, b.val)


In [ ]:


def Empty(T):
    return smt.K(T, smt.IntVal(0))



def Sum(a, b):
    return smt.Map( a, b)

def Union():
    return max
def Inter():
    return min

def Diff(a, b):
    # floor 0


card = smt.Function("card", )
card = 


Resource = smt.DeclareSort("Resource")
ResourceBag = Bag(Resource)

Prop = kd.NewType('Prop', smt.ArraySort(ResourceBag, smt.BoolSort()))

def Sep(p,q):
    res1, res2 = smt.Consts("res1 res2", ResourceBag)
    return smt.Lambda([res], kd.QExists([res1, res2], smt.And(Sum(res1,res2) == res), p.val[res1], q.val[res2]))



# Modal
https://en.wikipedia.org/wiki/Modal_logic#Frames_and_completeness

In [ ]:
World = smt.DeclareSort("World")
acc = smt.Function("acc", World, World, BoolSort())

w, v = amt.Consts("w v")
refl = kd.axiom(smt.ForAll([w], acc(w, w)))
symm = kd.axiom(smt.ForAll([w, v], smt.Implies(acc(w, v), acc(v, w))))
trans = kd.axiom(smt.ForAll([w,u,v], smt.Implies(smt.And(acc(w, u), acc(u, v)), acc(w, v))))
serial = kd.axiom(smt.ForAll([w], smt.Exists([u], acc(w,u))))
euclidean = kd.axiom(kd.QForAll([w,u,v], acc(w, u), acc(w,v), acc(u, v)))


K = []
D = [serial]
T = [refl]
B = [refl,symm]
S4 = [refl, trans]
S5 = [refl, euclidean]


# ctl
https://en.wikipedia.org/wiki/Computation_tree_logic#Semantics_of_CTL

# intuitonistinc

Reals a truth values.

Instead of single ereal
Maybve minimal logic could avoid ereal. Doesn't need a bottom

inifnite union.

Lex Positive. (0,0,0, eps, eps, eps) is still lex positive?
(eps,eps,eps, 0,0,0,0,...) is lex positive?



False => phi
uhhhhhh. phi - NegInt > 0
NegInf - NegInf > 0 is undef...


a > 0 is saying that a > eps1, a > eps2, a > eps3, ... must eventually become true.

In [ ]:
import kdrag.theories.real.extended as ereal
Prop = ereal.EReal #smt.RealSort()
# ereal is topologically the same as unit closed interval
TRUE = ereal.EReal.Inf
FALSE = ereal.EReal.NegInf

def And(x,y):
    return smt.If(x <= y, x, y)
def Or(x,y):
    return smt.If(x <= y, y, x)
def Implies(x,y):
    return y - x
def Models(phi):
    return phi > EReal.Fin(0) # in open set



# free logic


